In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import csv
from collections import defaultdict
import json
from lightfm import LightFM
from lightfm.data import Dataset
import optuna
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
import os
import numpy as np

# Default path to data files
PATH = "../data/"

In [2]:
# Load user-item interaction data
interaction_data = pd.read_csv(
    PATH + 'ml-100k/u.data',
    sep='\t',
    encoding="latin1",
    names=['user_id', 'item_id', 'rating', 'timestamp']
    )[['user_id', 'item_id', 'rating']]
display(interaction_data.shape)
interaction_data.head(5)

(100000, 3)

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [3]:
# Instantiate a defaultdict to hold user features
user_data = defaultdict(dict)

# Read data and build user features dictionary
def load_feature(file_path, feature_name):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            userId = row['userId']
            value = row[feature_name]
            user_data[userId][feature_name] = value

# Load each feature file
load_feature(PATH + 'ageRel.csv', 'age')
load_feature(PATH + 'genderRel.csv', 'gender')
load_feature(PATH + 'occupationRel.csv', 'occupation')
load_feature(PATH + 'residesRel.csv', 'zipcode')

# Build user features list
user_features_raw = [
    (userId, [f'age:{data["age"]}', f'gender:{data["gender"]}',
              f'occupation:{data["occupation"]}', f'zipcode:{data["zipcode"]}'])
    for userId, data in user_data.items()
]

# Display first 5 user features
for item in user_features_raw[:5]:
    print(item)

('1', ['age:24', 'gender:M', 'occupation:technician', 'zipcode:85'])
('2', ['age:53', 'gender:F', 'occupation:other', 'zipcode:94'])
('3', ['age:23', 'gender:M', 'occupation:writer', 'zipcode:32'])
('4', ['age:24', 'gender:M', 'occupation:technician', 'zipcode:43'])
('5', ['age:33', 'gender:F', 'occupation:other', 'zipcode:15'])


In [4]:
# Instantiate a defaultdict to hold item features
item_data = defaultdict(lambda: defaultdict(list))

# Read data and build item features dictionary
# Modified version to handle multiple genres
def load_feature(file_path, feature_name):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            itemId = row['movieId']
            value = row[feature_name]
            if feature_name == 'genreDesc':
                item_data[itemId]['genre'].append(value)
            else:
                item_data[itemId][feature_name] = value

# Load each feature file
load_feature(PATH + 'releaseRel.csv', 'releaseDate')
load_feature(PATH + 'genreRel.csv', 'genreDesc')

# Build item features list
item_features_raw = [
    (
        itemId,
        [f'releaseDate:{data["releaseDate"]}'] +
        [f'genre:{genre}' for genre in data['genre']]
    )
    for itemId, data in item_data.items()
]

# Display first 5 item features
for item in item_features_raw[:5]:
    print(item)

('2', ['releaseDate:Jan-1995', 'genre:Action', 'genre:Adventure', 'genre:Thriller'])
('4', ['releaseDate:Jan-1995', 'genre:Action', 'genre:Comedy', 'genre:Drama'])
('17', ['releaseDate:Feb-1996', 'genre:Action', 'genre:Comedy', 'genre:Crime', 'genre:Horror', 'genre:Thriller'])
('21', ['releaseDate:Feb-1996', 'genre:Action', 'genre:Adventure', 'genre:Comedy', 'genre:Musical', 'genre:Thriller'])
('22', ['releaseDate:Feb-1996', 'genre:Action', 'genre:Drama', 'genre:War'])


In [5]:
# Load test item IDs from the json file saved
# previously from Knowledge Graph Method
with open('../experiments/test_ids.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
# Extract test item IDs as integers
test_item_ids = [item['movieId'] for item in data]
print(f"items on test set: {len(test_item_ids)}")
display(test_item_ids[:5])

items on test set: 30


['159', '458', '679', '128', '658']

In [6]:
# Split data into train and test sets for later evaluation
# with LightFM the same way as done in Knowledge Graph Method

# Interaction data for training (excluding test items)
# Remover todas as interações dos itens de teste permite
# simular o cenário de recomendação de novos itens
train_interactions_df = interaction_data[
    ~interaction_data['item_id'].astype(str).isin(test_item_ids)]

# Interaction data for testing (only test items)
# Usado na etapa de avaliação como ground truth
test_interactions_df = interaction_data[
    interaction_data['item_id'].astype(str).isin(test_item_ids)]

# Item side features for testing (only testing items)
test_item_features = [item for item in item_features_raw if item[0] in test_item_ids]

In [7]:
# Build lightfm Dataset
dataset = Dataset()

# All unique user and item ids because LightFM needs all ids
# even if some items are only in the test set
user_ids = interaction_data['user_id'].astype(str).unique()
# Os itens em cold start precisam estar no dataset
item_ids = interaction_data['item_id'].astype(str).unique()

# Unique features from user and item features
user_feature_set = set(f for _, feats in user_features_raw for f in feats)
item_feature_set = set(f for _, feats in item_features_raw for f in feats)

# Partial fit
dataset.fit(
    users=user_ids,
    items=item_ids,
    user_features=user_feature_set,
    item_features=item_feature_set
)

# Build training matrices considering only training interactions
# so its possible to simulate cold-start for items in the test set
(interactions, weights) = dataset.build_interactions(
    [(str(row['user_id']), str(row['item_id']), row['rating']) for _, row in train_interactions_df.iterrows()]
)

user_features = dataset.build_user_features(user_features_raw)
item_features = dataset.build_item_features(item_features_raw)

In [8]:
# Implement hiperparameter tuning using Optuna
# to find the best parameters for LightFM model

# Define the objective function for optimization
def objective(trial):
    # Hiperparâmetros a otimizar
    no_components = trial.suggest_int('no_components', 20, 100)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    loss = trial.suggest_categorical('loss', ['logistic'])

    # Validação cruzada: média de N splits
    n_splits = 3
    scores = []

    for _ in range(n_splits):
        # Split data into train and validation sets
        train, valid = random_train_test_split(
            interactions, test_percentage=0.2)
        
        # Instantiate and train the model
        model = LightFM(
            no_components=no_components,
            learning_rate=learning_rate,
            loss=loss
        )

        model.fit(
            train,
            user_features=user_features,
            item_features=item_features,
            epochs=20,
            num_threads=4
        )

        # Evaluate the model using precision@k
        score = precision_at_k(
            model,
            valid,
            k=5,
            user_features=user_features,
            item_features=item_features
        ).mean()
        scores.append(score)
        
    return sum(scores) / len(scores)

In [9]:
# Run the optimization with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best parameters found
print("Best parameters found:")
print(study.best_params)

# Save the best parameters to a JSON file
os.makedirs("../experiments", exist_ok=True)
with open("../experiments/lightfm_best_params.json", "w", encoding="utf-8") as f:
    json.dump(
        {
            "best_params": study.best_params,
            "best_precision_at_5": study.best_value
        },
        f,
        indent=4
    )

[I 2025-10-21 09:55:17,832] A new study created in memory with name: no-name-557d4d65-4a9e-4809-b03c-a6f24e5ccfac
[I 2025-10-21 09:58:07,796] Trial 0 finished with value: 0.02654140256345272 and parameters: {'no_components': 56, 'learning_rate': 0.00021276288037436626, 'loss': 'logistic'}. Best is trial 0 with value: 0.02654140256345272.
[I 2025-10-21 10:00:17,255] Trial 1 finished with value: 0.0217268243432045 and parameters: {'no_components': 42, 'learning_rate': 0.0014694546937152652, 'loss': 'logistic'}. Best is trial 0 with value: 0.02654140256345272.
[I 2025-10-21 10:01:24,572] Trial 2 finished with value: 0.015511673875153065 and parameters: {'no_components': 21, 'learning_rate': 0.055534227485426996, 'loss': 'logistic'}. Best is trial 0 with value: 0.02654140256345272.
[I 2025-10-21 10:03:08,192] Trial 3 finished with value: 0.022049011662602425 and parameters: {'no_components': 31, 'learning_rate': 0.005517986906626521, 'loss': 'logistic'}. Best is trial 0 with value: 0.02654

Best parameters found:
{'no_components': 77, 'learning_rate': 0.00010027576106954592, 'loss': 'logistic'}


In [10]:
# Load the best parameters from the JSON file
with open("../experiments/lightfm_best_params.json", "r", encoding="utf-8") as f:
    best = json.load(f)
best_params = best["best_params"]

# Instantiate and train the final model with the best parameters
# and interactions from the entire training set
final_model = LightFM(
    no_components=best_params["no_components"],
    learning_rate=best_params["learning_rate"],
    loss=best_params["loss"]
)
final_model.fit(
    interactions,
    user_features=user_features,
    item_features=item_features,
    epochs=20,
    num_threads=4
)

In [11]:
# Predict the 50 most relevant users for each item in the test set

# Load all the user IDs
user_ids = list(user_ids)

# Build the item features matrix for the test set
test_item_features_matrix = dataset.build_item_features(test_item_features)

# Generate recommendations for each test item
# getting top 50 users for each item so its possible to compare
# with Knowledge Graph Method results on k = 10, 20 and 50
top_k = 50
recommendations = {}

# Mapeamento reverso do índice interno para o ID real do usuário
user_id_map = {v: k for k, v in dataset.mapping()[0].items()}

for item_id in test_item_ids:
    # Índice interno do item de teste
    item_internal_idx = dataset.mapping()[2][item_id]
    # Score para todos os usuários para este item
    scores = final_model.predict(
        user_ids=np.arange(len(user_ids)),
        item_ids=np.repeat(item_internal_idx, len(user_ids)),
        user_features=user_features,
        item_features=test_item_features_matrix
    )
    # Top 50 usuários (índices ordenados por score decrescente)
    top_users_idx = np.argsort(-scores)[:top_k]
    # IDs reais dos usuários
    top_users = [user_ids[i] for i in top_users_idx]
    recommendations[item_id] = top_users

In [12]:
# Calculate precision@k and ndcg@k for k = 10, 20, 50
def precision_at_k(recommended, relevant, k):
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    hits = sum([1 for user in recommended_k if user in relevant_set])
    return hits / k

def dcg_at_k(recommended, relevant, k):
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    return sum([1 / np.log2(idx + 2) if user in relevant_set else 0
                for idx, user in enumerate(recommended_k)])

def ndcg_at_k(recommended, relevant, k):
    dcg = dcg_at_k(recommended, relevant, k)
    ideal_dcg = sum([1 / np.log2(idx + 2) for idx in range(min(len(relevant), k))])
    return dcg / ideal_dcg if ideal_dcg > 0 else 0.0

ks = [10, 20, 50]
precision_scores = {k: [] for k in ks}
ndcg_scores = {k: [] for k in ks}

# Crie um dicionário: item_id -> lista de usuários relevantes (do test_interactions_df)
test_relevant = (
    test_interactions_df.groupby('item_id')['user_id']
    .apply(list)
    .to_dict()
)

for item_id, recommended_users in recommendations.items():
    # Converta ambos para string para garantir a comparação correta
    relevant_users = [str(u) for u in test_relevant.get(int(item_id), [])]
    recommended_users = [str(u) for u in recommended_users]
    for k in ks:
        prec = precision_at_k(recommended_users, relevant_users, k)
        ndcg = ndcg_at_k(recommended_users, relevant_users, k)
        precision_scores[k].append(prec)
        ndcg_scores[k].append(ndcg)

for k in ks:
    print(f"\nPrecision@{k} por item:")
    print(precision_scores[k])  # Lista com 30 valores

    print(f"NDCG@{k} por item:")
    print(ndcg_scores[k])


Precision@10 por item:
[0.1, 0.0, 0.1, 0.2, 0.3, 0.1, 0.1, 0.1, 0.2, 0.3, 0.3, 0.0, 0.0, 0.2, 0.1, 0.4, 0.3, 0.1, 0.0, 0.4, 0.1, 0.3, 0.1, 0.4, 0.4, 0.0, 0.1, 0.2, 0.1, 0.0]
NDCG@10 por item:
[np.float64(0.13886244387355454), np.float64(0.0), np.float64(0.07336392209936005), np.float64(0.21222636597291458), np.float64(0.2973694836145356), np.float64(0.22009176629808017), np.float64(0.22009176629808017), np.float64(0.07336392209936005), np.float64(0.21726071285222986), np.float64(0.2685529228900211), np.float64(0.2735872697693364), np.float64(0.0), np.float64(0.0), np.float64(0.21222636597291458), np.float64(0.11004588314904008), np.float64(0.40741536676357565), np.float64(0.3340514446642156), np.float64(0.07336392209936005), np.float64(0.0), np.float64(0.3757677525932109), np.float64(0.06362078819895171), np.float64(0.2758471541718663), np.float64(0.07336392209936005), np.float64(0.5173633627401372), np.float64(0.5231737964779628), np.float64(0.0), np.float64(0.11004588314904008), np.

In [13]:
# Média para cada k
for k in ks:
    print(f"Precision@{k}: {np.mean(precision_scores[k]):.4f}")
    print(f"NDCG@{k}: {np.mean(ndcg_scores[k]):.4f}")

Precision@10: 0.1667
NDCG@10: 0.1859
Precision@20: 0.1733
NDCG@20: 0.1819
Precision@50: 0.2173
NDCG@50: 0.2108


In [22]:
# Salvar listas completas de métricas por item
results = {
    **{f"precision@{k}": precision_scores[k] for k in ks},
    **{f"ndcg@{k}": ndcg_scores[k] for k in ks}
}

with open("../experiments/lightfm_final_metrics.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4)